In [1]:
from bs4 import BeautifulSoup as bs
from datetime import *
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from urllib.request import Request, urlopen
from lxml import html
import openpyxl
import pygame
import re
import time
import requests

In [2]:
def betweenCarats(test):
    info = []
    while(True):
        start = test.find('>')
        end = test[start:].find('<')
        if end == -1:
            # if the list is empty grab everything before the first <
            if len(info) == 0:
                info.append(test[:start - 3].strip())
            break
        add = test[start + 1:][:end - 1].strip()
        if add != "":
            info.append(add)
        test = test[start + end:]
    return info

In [3]:
PARCEL_ID                  = "A"
ADDRESS                    = "B"
PROPERTY_TYPE              = "C"
CLASSIFICATION_CODE        = "D"
LOT_SIZE                   = "E"
LIVING_AREA                = "F"
YEAR_BUILT                 = "G"
YEAR_REMODELED             = "H"
OWNER2018                  = "I"
OWNER_ADDRESS              = "J"
RESIDENTIAL_EXEMPTION      = "K"
PERSONAL_EXEMPTION         = "L"

BUILDING_VALUE_2018        = "M"
LAND_VALUE_2018            = "N"
TOTAL_VALUE_2018           = "O"

STYLE                      = "P" 
EXTERIOR_CONDITION         = "Q"
EXTERIOR_WALL              = "R"
GRADE                      = "S"
STORIES                    = "T"
NUMBER_OF_UNITS            = "U"
STREET_TYPE                = "V"
SQUARE_FEET_OF_LIVING_AREA = "W"
BASE_FLOOR                 = "X"
FIREPLACES                 = "Y"
NUMBER_OF_FLOORS           = "Z"
TOTAL_ROOMS                = "AA"
BEDROOMS                   = "AB"
BATHROOMS                  = "AC"
HALF_BATHROOMS             = "AD"
BATH_STYLE_1               = "AE"
BATH_STYLE_2               = "AF"
BATH_STYLE_3               = "AG"
KITCHEN_TYPE               = "AH"
HEAT_TYPE                  = "AI"
INTERIOR_CONDITION         = "AJ"
INTERIOR_FINISH            = "AK"
ORIENTATION                = "AL"
CORNER_UNIT                = "AM"
VIEW                       = "AN"

TYPES                      = "AO"
SQRFT                      = "AP"

FIRST_YEAR                 = 43


# Create a new excel file
out = openpyxl.Workbook()
# Open the worksheet we want to edit
outsheet = out.create_sheet("Property Info")

# if 'sheet' appears randomly we can delete it
rm = out.get_sheet_by_name('Sheet')
out.remove_sheet(rm)

outsheet[PARCEL_ID                  + '1'].value = "Parcel Id"
outsheet[ADDRESS                    + '1'].value = "Address"
outsheet[PROPERTY_TYPE              + '1'].value = "Property Type"
outsheet[CLASSIFICATION_CODE        + '1'].value = "Classification Code"
outsheet[LOT_SIZE                   + '1'].value = "Lot Size"
outsheet[LIVING_AREA                + '1'].value = "Living Area"
outsheet[YEAR_BUILT                 + '1'].value = "YEAR BUILT".title()
outsheet[YEAR_REMODELED             + '1'].value = "YEAR REMODELED".title()
outsheet[OWNER2018                  + '1'].value = "OWNER AS OF 2018".title()
outsheet[OWNER_ADDRESS              + '1'].value = "OWNER'S ADDRESS".title()
outsheet[RESIDENTIAL_EXEMPTION      + '1'].value = "RESIDENTIAL EXEMPTION".title()
outsheet[PERSONAL_EXEMPTION         + '1'].value = "PERSONAL EXEMPTION".title()

outsheet[BUILDING_VALUE_2018        + '1'].value = "BUILDING VALUE 2018".title()
outsheet[LAND_VALUE_2018            + '1'].value = "LAND VALUE 2018".title()
outsheet[TOTAL_VALUE_2018           + '1'].value = "TOTAL VALUE 2018".title()

outsheet[STYLE                      + '1'].value = "STYLE".title()
outsheet[EXTERIOR_CONDITION         + '1'].value = "EXTERIOR CONDITION".title()
outsheet[EXTERIOR_WALL              + '1'].value = "EXTERIOR WALL".title()
outsheet[GRADE                      + '1'].value = "GRADE".title()
outsheet[STORIES                    + '1'].value = "STORIES".title()
outsheet[NUMBER_OF_UNITS            + '1'].value = "NUMBER OF UNITS".title()
outsheet[STREET_TYPE                + '1'].value = "STREET TYPE".title()
outsheet[SQUARE_FEET_OF_LIVING_AREA + '1'].value = "SQUARE FEET OF LIVING_AREA".title()
outsheet[BASE_FLOOR                 + '1'].value = "BASE FLOOR".title()
outsheet[FIREPLACES                 + '1'].value = "FIREPLACES".title()
outsheet[NUMBER_OF_FLOORS           + '1'].value = "NUMBER OF FLOORS".title()
outsheet[TOTAL_ROOMS                + '1'].value = "TOTAL ROOMS".title()
outsheet[BEDROOMS                   + '1'].value = "BEDROOMS".title()
outsheet[BATHROOMS                  + '1'].value = "BATHROOMS".title()
outsheet[HALF_BATHROOMS             + '1'].value = "HALF BATHROOMS".title()
outsheet[BATH_STYLE_1               + '1'].value = "BATH STYLE 1".title()
outsheet[BATH_STYLE_2               + '1'].value = "BATH STYLE 2".title()
outsheet[BATH_STYLE_3               + '1'].value = "BATH STYLE 3".title()
outsheet[KITCHEN_TYPE               + '1'].value = "KITCHEN TYPE".title()
outsheet[HEAT_TYPE                  + '1'].value = "HEAT TYPE".title()
outsheet[INTERIOR_CONDITION         + '1'].value = "INTERIOR CONDITION".title()
outsheet[INTERIOR_FINISH            + '1'].value = "INTERIOR FINISH".title()
outsheet[ORIENTATION                + '1'].value = "ORIENTATION".title()
outsheet[CORNER_UNIT                + '1'].value = "CORNER UNIT".title()
outsheet[VIEW                       + '1'].value = "VIEW".title()

outsheet[TYPES                      + '1'].value = "Extra Features".title()
outsheet[SQRFT                      + '1'].value = "Size/Square Feet of Extra Features".title()

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:56: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:57: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).


In [6]:
# Get all unique parcel id's
url = 'https://www.cityofboston.gov/assessing/search/?'
address = 'q=otis+place'

info = []

browser = webdriver.PhantomJS(desired_capabilities={'phantomjs.page.settings.resourceTimeout': '5000'})

fullUrl = url + address

print(fullUrl)

browser.get(fullUrl)

HTML = browser.execute_script("return document.body.innerHTML")
# innerHTML = HTML.split('\n')

# print(HTML)

regexPID = 'pid=[\d]+'
ids = re.findall(regexPID, HTML)
uniques = []
for i in ids:
    if i not in uniques:
        uniques.append(i)

print(uniques)

browser.quit()

https://www.cityofboston.gov/assessing/search/?q=otis+place
['pid=1805036010', 'pid=0502518000', 'pid=0502522000', 'pid=1805038000', 'pid=0502522004', 'pid=0502522005', 'pid=0502522006', 'pid=0502522007', 'pid=0502522008', 'pid=0502522009', 'pid=0502522010', 'pid=0502522011', 'pid=0502522012', 'pid=0502522013', 'pid=0502521000', 'pid=1805037000', 'pid=0502520000', 'pid=0502519000', 'pid=0502519002', 'pid=0502519004', 'pid=0502519006', 'pid=0502519008', 'pid=0502519010', 'pid=1805035000', 'pid=0502527000', 'pid=0502526000', 'pid=0502526004', 'pid=0502526006', 'pid=0502526008', 'pid=0502526002', 'pid=0502525000', 'pid=1805040000', 'pid=0502524000', 'pid=1805036020', 'pid=0502524002', 'pid=0502524004', 'pid=0502524010', 'pid=0502523000', 'pid=1805039000']


In [7]:
url = 'https://www.cityofboston.gov/assessing/search/?'

for j in range(0, len(uniques)):
    parcel = uniques[j]
    
    fullUrl = url + parcel

    print(fullUrl)

    driver = webdriver.PhantomJS(desired_capabilities={'phantomjs.page.settings.resourceTimeout': '5000'})
    
    print('Opening browser...')
    driver.get(fullUrl)
    # browser.get(fullUrl)
    #HTML = requests.get(url)
    
    #print(HTML)
    #print(HTML.text)
    
    print("Capturing HTML...")
    HTML = driver.execute_script("return document.body.innerHTML")
    
    #print(HTML)
    
    outsheet[PARCEL_ID                  + str(j + 2)].value = "PARCEL_ID"
    
    regexNewLine = '\n'
    HTML = re.sub(regexNewLine, '', HTML)
    
    ###############
    # TOP SECTION #
    ###############
    
    print("Getting top section info...")
    regexInfo = '>Address:<.*?right">(.*?)<.*?>Property Type:<.*?right">(.*?)<.*?>Classification Code:<.*?right">(.*?)<.*?>Lot Size:<.*?right">(.*?)<.*?>Living Area:<.*?right">(.*?)<.*?>Year Built:<.*?right">(.*?)<.*?>Year Remodeled:<.*?right">(.*?)<.*?>Owner on Monday, January 1, 2018:<.*?owner=(.*?)".*?>Owner\'s Mailing Address:<.*?right">(.*?)<.*?>Residential Exemption:<.*?right">(.*?)<.*?>Personal Exemption:<.*?right">(.*?)<.*?'
    matchInfo = re.search(regexInfo, HTML)
    
    if matchInfo:
        print('Address:                   ', matchInfo.group(1))
        print('Property type:             ', matchInfo.group(2))
        print('Classification:            ', matchInfo.group(3))
        print('Lot Size:                  ', matchInfo.group(4))
        print('Living Area:               ', matchInfo.group(5))
        print('Year Build:                ', matchInfo.group(6))
        print('Year Remodeled:            ', matchInfo.group(7))
        print('Owner:                     ', matchInfo.group(8))
        print('Owner Address:             ', matchInfo.group(9))
        print('Residential Ex:            ', matchInfo.group(10))
        print('Personal Ex:               ', matchInfo.group(11))

        outsheet[ADDRESS                    + str(j + 2)].value = matchInfo.group(1).strip().title()
        outsheet[PROPERTY_TYPE              + str(j + 2)].value = matchInfo.group(2).strip()
        outsheet[CLASSIFICATION_CODE        + str(j + 2)].value = matchInfo.group(3).strip()
        outsheet[LOT_SIZE                   + str(j + 2)].value = matchInfo.group(4).strip()
        outsheet[LIVING_AREA                + str(j + 2)].value = matchInfo.group(5).strip()
        outsheet[YEAR_BUILT                 + str(j + 2)].value = matchInfo.group(6).strip()
        outsheet[YEAR_REMODELED             + str(j + 2)].value = matchInfo.group(7).strip()
        outsheet[OWNER2018                  + str(j + 2)].value = matchInfo.group(8).strip().title()
        outsheet[OWNER_ADDRESS              + str(j + 2)].value = matchInfo.group(9).strip().title()
        outsheet[RESIDENTIAL_EXEMPTION      + str(j + 2)].value = matchInfo.group(10).strip()
        outsheet[PERSONAL_EXEMPTION         + str(j + 2)].value = matchInfo.group(11).strip()

    ###############
    # TAX SECTION #
    ###############
    
    print("Getting tax info...")
    regexInfo = '>FY2018 Building value:<.*?(\$[\d\.,]+).*?>FY2018 Land Value:<.*?(\$[\d\.,]+).*?>FY2018 Total Assessed Value:<.*?(\$[\d\.,]+)'
    matchInfo = re.search(regexInfo, HTML)
    
    if matchInfo:
        print()
        print('Building Value:            ', matchInfo.group(1))
        print('Land Value:                ', matchInfo.group(2))
        print('Total Value:               ', matchInfo.group(3))

        outsheet[BUILDING_VALUE_2018        + str(j + 2)].value = matchInfo.group(1)
        outsheet[LAND_VALUE_2018            + str(j + 2)].value = matchInfo.group(2)
        outsheet[TOTAL_VALUE_2018           + str(j + 2)].value = matchInfo.group(3)
    
    ####################
    # BUILDING SECTION #
    ####################

    print("Getting building info...")
    regexInfo = '>Style:<.*?right">(.*?)<.*?>Exterior Condition:<.*?right">(.*?)<.*?>Exterior Wall:<.*?right">(.*?)<.*?>Grade:<.*?right">(.*?)<.*?>Stories:<.*?right">(.*?)<.*?>Number of Units:<.*?right">(.*?)<.*?>Street Type:<.*?right">(.*?)<.*?>Square Feet of Living Area:<.*?right">(.*?)<.*?>Base Floor:<.*?right">(.*?)<.*?>Fireplaces:<.*?right">(.*?)<.*?>Number of Floors:<.*?right">(.*?)<.*?>Total Rooms:<.*?right">(.*?)<.*?>Bedrooms:<.*?right">(.*?)<.*?>Bathrooms:<.*?right">(.*?)<.*?>Half Bathrooms:<.*?right">(.*?)<.*?>Bath Style 1:<.*?right">(.*?)<.*?>Bath Style 2:<.*?right">(.*?)<.*?>Bath Style 3:<.*?right">(.*?)<.*?>Kitchen Type:<.*?right">(.*?)<.*?>Heat Type:<.*?right">(.*?)<.*?>Interior Condition:<.*?right">(.*?)<.*?>Interior Finish:<.*?right">(.*?)<.*?>Orientation:<.*?right">(.*?)<.*?>Corner Unit:<.*?right">(.*?)<.*?>View:<.*?right">(.*?)<.*?'
    matchInfo = re.search(regexInfo, HTML)
    
    if matchInfo:
        print()
        print('Style:                     ', matchInfo.group(1))
        print('Exterior Condition:        ', matchInfo.group(2))
        print('Exterior Wall:             ', matchInfo.group(3))
        print('Grade:                     ', matchInfo.group(4))
        print('Stories:                   ', matchInfo.group(5))
        print('Number of Units:           ', matchInfo.group(6))
        print('Street Type:               ', matchInfo.group(7))
        print('Square Feet of Living Area:', matchInfo.group(8))
        print('Base Floor:                ', matchInfo.group(9))
        print('Fireplaces:                ', matchInfo.group(10))
        print('Number of Floors:          ', matchInfo.group(11))
        print('Total Rooms:               ', matchInfo.group(12))
        print('Bedrooms:                  ', matchInfo.group(13))
        print('Bathrooms:                 ', matchInfo.group(14))
        print('Half Bathrooms:            ', matchInfo.group(15))
        print('Bath Style 1:              ', matchInfo.group(16))
        print('Bath Style 2:              ', matchInfo.group(17))
        print('Bath Style 3:              ', matchInfo.group(18))
        print('Kitchen Type:              ', matchInfo.group(19))
        print('Heat Type:                 ', matchInfo.group(20))
        print('Interior Condition:        ', matchInfo.group(21))
        print('Interior Finish:           ', matchInfo.group(22))
        print('Orientation:               ', matchInfo.group(23))
        print('Corner Unit:               ', matchInfo.group(24))
        print('View:                      ', matchInfo.group(25))


        outsheet[STYLE                      + str(j + 2)].value = matchInfo.group(1)
        outsheet[EXTERIOR_CONDITION         + str(j + 2)].value = matchInfo.group(2)
        outsheet[EXTERIOR_WALL              + str(j + 2)].value = matchInfo.group(3)
        outsheet[GRADE                      + str(j + 2)].value = matchInfo.group(4)
        outsheet[STORIES                    + str(j + 2)].value = matchInfo.group(5)
        outsheet[NUMBER_OF_UNITS            + str(j + 2)].value = matchInfo.group(6)
        outsheet[STREET_TYPE                + str(j + 2)].value = matchInfo.group(7)
        outsheet[SQUARE_FEET_OF_LIVING_AREA + str(j + 2)].value = matchInfo.group(8)
        outsheet[BASE_FLOOR                 + str(j + 2)].value = matchInfo.group(9)
        outsheet[FIREPLACES                 + str(j + 2)].value = matchInfo.group(10)
        outsheet[NUMBER_OF_FLOORS           + str(j + 2)].value = matchInfo.group(11)
        outsheet[TOTAL_ROOMS                + str(j + 2)].value = matchInfo.group(12)
        outsheet[BEDROOMS                   + str(j + 2)].value = matchInfo.group(13)
        outsheet[BATHROOMS                  + str(j + 2)].value = matchInfo.group(14)
        outsheet[HALF_BATHROOMS             + str(j + 2)].value = matchInfo.group(15)
        outsheet[BATH_STYLE_1               + str(j + 2)].value = matchInfo.group(16)
        outsheet[BATH_STYLE_2               + str(j + 2)].value = matchInfo.group(17)
        outsheet[BATH_STYLE_3               + str(j + 2)].value = matchInfo.group(18)
        outsheet[KITCHEN_TYPE               + str(j + 2)].value = matchInfo.group(19)
        outsheet[HEAT_TYPE                  + str(j + 2)].value = matchInfo.group(20)
        outsheet[INTERIOR_CONDITION         + str(j + 2)].value = matchInfo.group(21)
        outsheet[INTERIOR_FINISH            + str(j + 2)].value = matchInfo.group(22)
        outsheet[ORIENTATION                + str(j + 2)].value = matchInfo.group(23)
        outsheet[CORNER_UNIT                + str(j + 2)].value = matchInfo.group(24)
        outsheet[VIEW                       + str(j + 2)].value = matchInfo.group(25)

    ##################
    # EXTRA FEATURES #
    ##################
    
    print("Getting extra features info...")
    regexInfo = '>Type:<.*?right">(.*?)<.*?>Size/sqft:<.*?right">(.*?)<'
    matchInfo = re.findall(regexInfo, HTML)
    
    print()
    print("Extra features:            ", matchInfo)
    
    if len(matchInfo) > 0:
        types, sqrft = "", ""
        for i in range (0, len(matchInfo)):
            temp1, temp2 = matchInfo[i]
            types = types + str(temp1) + '; '
            sqrft = sqrft + str(temp2) + '; '
        
        types = types[:-2]
        sqrft = sqrft[:-2]
        
        outsheet[TYPES                      + str(j + 2)].value = types
        outsheet[SQRFT                      + str(j + 2)].value = sqrft
    else:
        outsheet[TYPES                      + str(j + 2)].value = "None"
        outsheet[SQRFT                      + str(j + 2)].value = "N/A"
    
    #################
    # VALUE HISTORY #
    #################
    
    print("Getting value history info...")
    regexValue = 'Fiscal Year(.*)'
    matchValue = re.search(regexValue, HTML)
    if matchValue:
        regexInfo = 'center">(\d{4})<.*?center">(.*?)<.*?center">([\$\d\.,]+)<'
        matchInfo = re.findall(regexInfo, matchValue.group(1))
        
        print(matchInfo)
        
        # for each match we need to create headers and populate columns
        for i in range(0, len(matchInfo)):
            year, property_type, value = matchInfo[i]
            outsheet.cell(row=1,column=FIRST_YEAR + (i * 2)).value = year + " Property Type"
            outsheet.cell(row=1,column=FIRST_YEAR + (i * 2) + 1).value = year + " Assessed Value"
    
            outsheet.cell(row=j + 2,column=FIRST_YEAR + (i * 2)).value = property_type
            outsheet.cell(row=j + 2,column=FIRST_YEAR + (i * 2) + 1).value = value
    
    print("Saving...")
    out.save("taxInfo.xlsx")
    
    print("Closing...")
    driver.close()

https://www.cityofboston.gov/assessing/search/?pid=1805036010
Opening browser...
Capturing HTML...
Getting top section info...
Address:                     OTIS PL BOSTON MA 02131
Property type:              Residential Land
Classification:             106 (Residential Property  / RES ANCILL IMPROVEMT)
Lot Size:                   1,350 sq ft
Living Area:                0 sq ft
Year Build:                 -
Year Remodeled:             -
Owner:                      DAGDIGIAN  CHRISTOPHER  R
Owner Address:              4394 WASHINGTON ST ROSLINDALE MA 02131
Residential Ex:             No
Personal Ex:                No
Getting tax info...

Building Value:             $0.00
Land Value:                 $23,800.00
Total Value:                $23,800.00
Getting building info...
Getting extra features info...

Extra features:             []
Getting value history info...
[('2018', 'Residential Land', '$23,800.00'), ('2017', 'Residential Land', '$23,300.00'), ('2016', 'Residential Land', '$22,600

KeyboardInterrupt: 